In [2]:
import ollama
import json
import requests 
from bs4 import BeautifulSoup
from IPython.display import display, Markdown, update_display
from dotenv import load_dotenv
from openai import OpenAI
import os

In [3]:
MODEL_LLAMA = 'llama3.2' 
MODEL_GPT = 'gpt-4o-mini'
MODEL_MISTRAL = 'mistral'
MODEL_QWEN = 'qwen2.5'

In [4]:
HEADERS = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        self.url = url
        response = requests.get(url=self.url,headers=HEADERS)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else 'Title not found'
        if soup.body:
            for irr in soup.body(['script','style','img','input']):
                irr.decompose()
            self.text = soup.body.get_text(separator='\n', strip=True)
        else:
            self.text = ''
        
        self.links = [link.get('href') for link in soup.find_all('a') if link]

    def get_content(self):
        return f'\nWebpage title:\n{self.title}\nWebpage Content:\n{self.text}'

In [63]:
ref_guru = Website('https://refactoring.guru/')
print(ref_guru.links)

['/store', '#checkout', '#checkout', '/sendy/form', 'https://www.facebook.com/refactoring.guru/', 'https://www.facebook.com/refactoring.guru/', '/refactoring', '/refactoring', '/design-patterns', '/design-patterns', '/', '/store', '/design-patterns/book', '/refactoring/course', '/refactoring', '/refactoring/what-is-refactoring', '/refactoring/what-is-refactoring', '/refactoring/technical-debt', '/refactoring/when', '/refactoring/how-to', '/refactoring/catalog', '/refactoring/smells', '/refactoring/smells/bloaters', '/smells/long-method', '/smells/large-class', '/smells/primitive-obsession', '/smells/long-parameter-list', '/smells/data-clumps', '/refactoring/smells/oo-abusers', '/smells/switch-statements', '/smells/temporary-field', '/smells/refused-bequest', '/smells/alternative-classes-with-different-interfaces', '/refactoring/smells/change-preventers', '/smells/divergent-change', '/smells/shotgun-surgery', '/smells/parallel-inheritance-hierarchies', '/refactoring/smells/dispensables'

# First Part: Feeding links to model and fetching relevant links

In [5]:
def set_prompt_forlink(website):
    system_prompt = """
    You are a decision maker provided with webpage links for single online learning resource website,
    Your job is to analyze different webpage links and decide which links have crucial learning material,
    Avoid links unrelated to learning like - 'about us', 'contact us', etc.
    Provide output in below JSON format example:\n
    {
        'links':[
            {'type':'What's a design pattern?', 'url':'https/complete/the/url/example-1'},
            {'type':'Catalog', 'url':'https/complete/the/url/example-2'},
        ]
    }
    """

    user_prompt = f'Link for the online resource website: {website.url} - title: {website.title}'
    user_prompt += """
    Decide which links have learning resources and discard unrelated links.
    kindly provide the complete https urls in output with desired format.
    """
    user_prompt +='here are the webpage links associated with the website - (pick the learning related links)'
    user_prompt += '\n'.join(website.links)
    
    return [
        {
            'role':'system',
            'content':system_prompt
        },
        {
            'role':'user',
            'content':user_prompt
        }
    ]


In [7]:
set_prompt_forlink(ref_guru)

[{'role': 'system',
  'content': "\n    You are a decision maker provided with webpage links for single online learning resource website,\n    Your job is to analyze different webpage links and decide which links have crucial learning material,\n    Avoid links unrelated to learning like - 'about us', 'contact us', etc.\n    Provide output like below JSON example:\n\n    {\n        'links':{\n            {'type':'What's a design pattern?', 'url':'https/complete/the/url/example-1'},\n            {'type':'Catalog', 'url':'https/complete/the/url/example-2'},\n        }\n    }\n    "},
 {'role': 'user',
  'content': 'Link for the online resource website: https://refactoring.guru/ - title: Refactoring and Design Patterns\n    Decide which links have learning resources and discard unrelated links.\n    kindly provide the complete https urls in output with desired format.\n    here are the webpage links associated with the website - (many might be relevant)/store\n#checkout\n#checkout\n/sendy

In [6]:
def printer(data):
    display(Markdown(data))

In [13]:
def call_llama_for_links(website, model):
    llama_response = ollama.chat(
        model=model,
        messages=set_prompt_forlink(website),
        format='json'
    )

    return json.loads(llama_response['message']['content'])

# printer(call_llama_for_links(ref_guru))
# links = call_llama_for_links(ref_guru)


In [59]:
print(f"Dict : {isinstance(links['links'][0]['type'],dict)}")
print(f"String : {isinstance(links['links'][0]['type'],str)}")
print(f"List : {isinstance(links['links'][0]['type'],list)}")
# print([link['type'] for link in links['links']])
# print([link['url'] for link in links['links']])

Dict : False
String : True
List : False


# Second Part: Feeding learning data to model

In [14]:
def get_all_data(website, model):
    result = 'Landing Page\n'
    result += website.get_content()
    links = call_llama_for_links(website, model)
    # print(f'Links found:\n {links}')
    for link in links['links']:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_content()
    return result

In [15]:
def set_prompt_for_learning(website, model):
    system_prompt = 'You are a tutor, provided with resources from a learning website.\
    Make a detailed course out of that material provided to you\
    Such that the student (user) can understand and master the concepts from the resources.\
    Also give examples and make assignments for the course.\
    Response should be in markdown.'

    user_prompt = f'I want help understanding and mastering these concepts from this website - {website.title}\n'
    user_prompt += 'please make an extensive course from this material provided to you, scrap unecessary information from the resource that will not help in this course.\n'
    user_prompt += 'The course should cover all the topics and kindly curate some easy to understand examples and assigments for practice.\n'
    user_prompt += 'Provide a complete clarification for all main and sub topics in one response only, dont wait for me to ask for it'
    user_prompt += f'Below is the full resource data scraped from the learning website.\n {get_all_data(website, model)}'

    return [
        {
            'role':'system',
            'content':system_prompt
        },
        {
            'role':'user',
            'content':user_prompt
        }
    ]

In [64]:
print(set_prompt_for_learning(ref_guru))

[{'role': 'system', 'content': 'You are a tutor, provided with resources from a learning website.    Make a detailed course out of that material provided to you    Such that the student (user) can understand and master the concepts from the resources.    Also give examples and make assignments for the course.    Response should be in markdown.'}, {'role': 'user', 'content': 'I want help understanding and mastering these concepts from this website - Refactoring and Design Patterns\nplease make an extensive course from this material provided to you, scrap unecessary information from the resource that will not help in this course.\nThe course should cover all the topics and kindly curate some easy to understand examples and assigments for practice.\nProvide a complete clarification for all main and sub topics in one response only, dont wait for me to ask for itBelow is the full resource data scraped from the learning website.\n Landing Page\n\nWebpage title:\nRefactoring and Design Patter

In [16]:
def call_llama_for_learning(website, model):
    stream = ollama.chat(
        model=MODEL_LLAMA,
        messages=set_prompt_for_learning(website, model),
        stream=True
    )

    response = ''
    display_handle = display(Markdown(''), display_id=True)
    for chunks in stream:
        response += chunks['message']['content'] or ''
        # response += response.replace('```','').replace('markdown','')
        update_display(Markdown(response),display_id=display_handle.display_id)


In [65]:
call_llama_for_learning(ref_guru)

It seems like you're trying to access the Refactoring.Guru website, but I'll provide a summary of the content instead.

Refactoring.Guru is a platform that offers resources and knowledge on software development, refactoring, and design patterns. The website provides articles, tutorials, and forums for discussing various topics related to software development.

You can sign in or register on the website if you don't have an account already.

Some popular topics and resources on Refactoring.Guru include:

1. **Design Patterns**: The website covers a wide range of design patterns, including Creational, Structural, and Behavioral patterns.
2. **Refactoring**: There are articles and tutorials on various refactoring techniques, such as extracting methods, inlining functions, and simplifying code.
3. **Programming Languages**: Refactoring.Guru has resources dedicated to specific programming languages, like Java, Python, JavaScript, and more.

If you have any specific questions or topics you'd like to discuss, feel free to ask, and I'll do my best to help!

In [17]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
openai = OpenAI()

In [18]:
def call_gpt_for_learning(website, model):
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=set_prompt_for_learning(website, model),
        stream=True
    )

    response = ''
    display_handle = display(Markdown(''), display_id=True)
    for chunks in stream:
        response += chunks.choices[0].delta.content or ''
        # response += response.replace('```','').replace('markdown','')
        update_display(Markdown(response),display_id=display_handle.display_id)


In [71]:
call_gpt_for_learning(ref_guru)

# Comprehensive Course on Refactoring and Design Patterns

This course aims to provide a thorough understanding and mastery of refactoring and design patterns, leveraging the materials from the outlined resource. Each section includes explanations, examples, and practical assignments to reinforce learning.

---

## Table of Contents

1. [Introduction to Refactoring](#introduction-to-refactoring)
2. [Design Patterns Overview](#design-patterns-overview)
3. [Design Patterns Categories](#design-patterns-categories)
4. [Creational Patterns](#creational-patterns)
   - [Factory Method](#factory-method)
   - [Abstract Factory](#abstract-factory)
   - [Builder](#builder)
   - [Prototype](#prototype)
   - [Singleton](#singleton)
5. [Structural Patterns](#structural-patterns)
   - [Adapter](#adapter)
   - [Bridge](#bridge)
   - [Composite](#composite)
   - [Decorator](#decorator)
   - [Facade](#facade)
   - [Flyweight](#flyweight)
   - [Proxy](#proxy)
6. [Behavioral Patterns](#behavioral-patterns)
   - [Chain of Responsibility](#chain-of-responsibility)
   - [Command](#command)
   - [Iterator](#iterator)
   - [Mediator](#mediator)
   - [Memento](#memento)
   - [Observer](#observer)
   - [State](#state)
   - [Strategy](#strategy)
   - [Template Method](#template-method)
   - [Visitor](#visitor)

---

## Introduction to Refactoring

**Definition**: Refactoring is the process of restructuring existing computer code—changing the factoring—without changing its external behavior.

### Key Concepts
- **Clean Code**: Writing code that is easy to understand and maintain.
- **Technical Debt**: The eventual costs incurred when choosing an easy solution now instead of using a better approach that would take longer.
- **When to Refactor**: Refactor when you notice code smells, like long methods, duplicated code, or complex conditionals.

### Assignment
- Identify a piece of code in one of your projects that could benefit from refactoring. List the code smell types present and propose a refactoring plan.

---

## Design Patterns Overview

**Definition**: Design patterns are typical solutions to commonly occurring problems in software design.

### Purpose of Design Patterns
- To provide reusable solutions to common problems within a given context in software design.
  
---

## Design Patterns Categories

Design patterns are categorized into three main types:
1. **Creational Patterns**: Deal with object creation mechanisms.
2. **Structural Patterns**: Deal with object composition and typically involve relationships between objects.
3. **Behavioral Patterns**: Deal with the interactions between objects.

---

## Creational Patterns

### Factory Method

**Intent**: Provides an interface for creating objects in a superclass, allowing subclasses to alter the type of objects that will be created.

#### Example
```python
class Transport:
    def deliver(self):
        pass

class Truck(Transport):
    def deliver(self):
        return "Deliver by land in a truck"

class Ship(Transport):
    def deliver(self):
        return "Deliver by sea in a ship"

class Logistics:
    def create_transport(self):
        pass

class RoadLogistics(Logistics):
    def create_transport(self):
        return Truck()

class SeaLogistics(Logistics):
    def create_transport(self):
        return Ship()
```

### Assignment
- Implement a Factory Method pattern in a small application of your choice.

### Abstract Factory

**Intent**: Lets you produce families of related objects without specifying their concrete classes.

#### Example
```python
class FurnitureFactory:
    def create_chair(self):
        pass
    
    def create_table(self):
        pass

class ModernFurnitureFactory(FurnitureFactory):
    def create_chair(self):
        return ModernChair()
    def create_table(self):
        return ModernTable()
```

### Assignment
- Create an Abstract Factory implementation for a furniture store, including at least two types of furniture.

### Builder

**Intent**: Separates the construction of a complex object from its representation so that the same construction process can create different representations.

### Prototype

**Intent**: Lets you copy existing objects without making your code dependent on their classes.

### Singleton

**Intent**: Ensures a class has only one instance and provides a global point of access to it.

---

## Structural Patterns

### Adapter

**Intent**: Allows incompatible interfaces to work together.

#### Example
```python
class EuropeanSocketInterface:
    def connect(self):
        return "Connected to European socket"

class USASocketInterface:
    def connect(self):
        return "Connected to USA socket"

class Adapter(EuropeanSocketInterface):
    def __init__(self, usa_socket):
        self.usa_socket = usa_socket

    def connect(self):
        return self.usa_socket.connect()
```

### Assignment
- Create an Adapter for a service that needs to interact with an incompatible system.

### Bridge

**Intent**: Split a large class or a set of closely related classes into two separate hierarchies—abstraction and implementation.

### Composite

**Intent**: Lets you compose objects into tree structures to represent part-whole hierarchies.

### Decorator

**Intent**: Adds new responsibilities to an object dynamically.

### Facade

**Intent**: Provides a simplified interface to a complex subsystem.

### Flyweight

**Intent**: Enables sharing of objects to support a large number of fine-grained objects efficiently.

### Proxy

**Intent**: Provides a surrogate or placeholder for another object to control access to it.

---

## Behavioral Patterns

### Chain of Responsibility

**Intent**: Lets you pass requests along a chain of handlers, allowing each handler to either process the request or pass it on.

### Command

**Intent**: Turns a request into a stand-alone object that contains all information about the request.

### Iterator

**Intent**: Lets you traverse elements of a collection without exposing its underlying representation.

### Mediator

**Intent**: Reduces chaotic dependencies between objects by letting them communicate only through a mediator object.

### Memento

**Intent**: Lets you save and restore the previous state of an object without revealing the details of its implementation.

### Observer

**Intent**: Lets you define a subscription mechanism to notify multiple objects about any events that happen to the object they’re observing.

### State

**Intent**: Lets an object alter its behavior when its internal state changes.

### Strategy

**Intent**: Lets you define a family of algorithms, encapsulate each one, and make them interchangeable.

### Template Method

**Intent**: Defines the skeleton of an algorithm in the superclass and lets subclasses override specific steps.

### Visitor

**Intent**: Lets you separate algorithms from the objects on which they operate.

---

## Conclusion and Final Assignment
Reflect on the concepts covered throughout this course. Choose a project where you can apply at least three different design patterns and document your thought process in applying them.

---

This course provides a foundation in refactoring and design patterns with practical examples and assignments aimed at reinforcing understanding and implementation in real-world scenarios.

In [11]:
git_convention = Website('https://www.conventionalcommits.org/en/v1.0.0/')

In [12]:
print(git_convention.links)

['/', '/en/v1.0.0', '/en/v1.0.0-beta.4', '/en/v1.0.0-beta.3', '/en/v1.0.0-beta.2', '/en/v1.0.0-beta.1', '/en/v1.0.0-beta', '/en/', '/it/', '/pl/', '/zh-hans/', '/zh-hant/', '/es/', '/ru/', '/ja/', '/fr/', '/ko/', '/pt-br/', '/id/', '/hy/', '/de/', '/th/', '/uk/', '/be/', '/tr/', '/nl/', '/ml/', '/ro/', '/bn/', '/uz/', '/ar/', '/en/about', '#summary', '#specification', 'https://github.com/conventional-commits/conventionalcommits.org', 'http://semver.org', 'http://semver.org/#summary', 'http://semver.org/#summary', 'http://semver.org/#summary', 'https://github.com/conventional-changelog/commitlint/tree/master/%40commitlint/config-conventional', 'https://github.com/angular/angular/blob/22b96b9/CONTRIBUTING.md#-commit-message-guidelines', 'https://git-scm.com/docs/git-interpret-trailers', 'https://www.ietf.org/rfc/rfc2119.txt', 'https://git-scm.com/docs/git-interpret-trailers', 'https://creativecommons.org/licenses/by/3.0/', 'https://www.netlify.com', 'https://github.com/conventional-commi

In [19]:
call_gpt_for_learning(git_convention, MODEL_QWEN)

# Course on Understanding and Mastering Conventional Commits

## Course Overview
This course is designed to teach you about Conventional Commits—a specification that aims to add human and machine-readable meaning to commit messages. You will learn the structure, types, footers, and benefits of using Conventional Commits. By the end of this course, you should be comfortable implementing this specification in your projects.

## Course Outline

### 1. Introduction to Conventional Commits
- **What are Conventional Commits?**
  - A lightweight convention on top of commit messages.
  - Facilitates explicit commit history and automation tools.
  
- **Relation to Semantic Versioning (SemVer)**
  - Describes features, fixes, and breaking changes in commits.
  
### 2. Structure of Commit Messages
- **Basic Structure**
  ```
  <type>[optional scope]: <description>

  [optional body]

  [optional footer(s)]
  ```
- **Elements Explained:**
  - `type`: Specifies the nature of change (e.g., `feat`, `fix`).
  - `scope`: Optional; specifies a section of the codebase (e.g., `parser`).
  - `description`: A concise description of the change.
  - `body`: Optional; provides further details.
  - `footer`: Optional; includes additional information like breaking changes.

### 3. Types of Commits
- **Core Types:**
  - `fix`: Bug fix (correlates with PATCH in SemVer).
  - `feat`: New feature (correlates with MINOR in SemVer).
  - `BREAKING CHANGE`: Indicates breaking API change.
  
- **Additional Types (recommended but optional)**
  - `build`, `chore`, `ci`, `docs`, `style`, `refactor`, `perf`, `test`.

### 4. Breaking Changes
- **Identification**
  - Use `!` after the type/scope or mention `BREAKING CHANGE:` in the footer.
  
- **Examples:**
  ```
  feat(api)!: send email upon shipment
  BREAKING CHANGE: The API endpoint for sending email has changed.
  ```

### 5. Footers in Commit Messages
- **Format**
  - Must have the format: `<word-token>: <value>` or `<word-token> #<issue number>`.
  
- **Examples**
  - `BREAKING CHANGE: <detail>`
  - `Reviewed-by: Z`
  - `Refs: #123`

### 6. Why Use Conventional Commits?
- Automatically generate CHANGELOGs.
- Determine semantic version bump based on commit types.
- Improve communication regarding changes in the codebase.
- Facilitate contributions from others.

### 7. Common Questions & FAQ
- **Q: How do I handle initial development phase commits?**
  - Treat them as production-ready for transparency.
  
- **Q: What if I used the wrong commit type?**
  - Use `git rebase -i` before merging to correct it.

- **Q: Must all contributors follow Conventional Commits?**
  - No, but it's beneficial. Squash commits can be cleaned up during merges.

## Examples of Commit Messages
1. Simple commit:
   ```
   fix: correct spelling in README.md
   ```
   
2. Commit with breaking change:
   ```
   feat!: drop support for Node 6
   BREAKING CHANGE: Environment variables now take precedence over config files.
   ```

3. Commit with multiple paragraphs:
   ```
   fix: prevent racing of requests

   Introduce a request ID to keep track of latest requests and dismiss others.
   Removed obsolete timeouts.

   Reviewed-by: A
   Refs #456
   ```

## Assignments
### Assignment 1: Crafting Commit Messages
Create a repository and make at least five commits using Conventional Commits.
- Include at least 2 breaking changes.
- Use at least 3 different types.

### Assignment 2: Analyzing Existing Commits
Find an open-source project and examine its commit history.
- Identify the types used.
- Report any inconsistencies with the Conventional Commit specification.

### Assignment 3: Create a CHANGLOG.md
Based on your commit history from Assignment 1, generate a CHANGLOG.md that summarizes all changes over versions based on Semantic Versioning.

### Assignment 4: Implement in a Team Project
Collaborate with others on a small project (can be a mock project) and ensure that all commits adhere to the Conventional Commits specification during development.

## Conclusion
By following this course, you will understand how to effectively apply Conventional Commits to improve project organization and communication. You will be equipped to implement this specification in your projects, thus benefiting your personal and collaborative development efforts. Good luck!

In [20]:
call_gpt_for_learning(git_convention, MODEL_MISTRAL)

# Course on Conventional Commits

## Course Overview
This course is designed to help you understand and master the concepts of Conventional Commits, a lightweight convention on commit messages used widely in software development. Utilizing the materials provided, we will cover:

1. **Introduction to Conventional Commits**
2. **Structure of Commit Messages**
3. **Types of Conventional Commits**
4. **Breaking Changes**
5. **Footers and Additional Metadata**
6. **Using Conventional Commits with Semantic Versioning**
7. **Best Practices and Advantages**
8. **Assignments and Practice Exercises**

---

## 1. Introduction to Conventional Commits
**Conventional Commits** is a specification that aims to standardize commit messages to provide a clear and meaningful commit history. This helps both humans and tools to understand the changes made over time.

### Key Benefits:
- Easier automated changelog generation
- Clearer project history for contributors
- Seamless integration with Continuous Integration (CI) and Continuous Deployment (CD) systems

---

## 2. Structure of Commit Messages
A commit message should be structured in the following format:

```
<type>[optional scope]: <description>

[optional body]

[optional footer(s)]
```

### Breakdown:
- **Type**: Specifies the nature of the commit (e.g., feat, fix).
- **Scope** (optional): Provides additional context (e.g., the area of impact).
- **Description**: A short and imperative explanation of the change.
- **Body** (optional): Provides more detailed information about the commit.
- **Footer(s)** (optional): Contains additional metadata (e.g., references to issues).

### Example:
```
feat(api): add support for fetching user data

BREAKING CHANGE: The user data request endpoint has changed.
```

---

## 3. Types of Conventional Commits
### Common Types:
1. **feat**: Introduces a new feature.
2. **fix**: Patches a bug.
3. **build**: Changes related to the build system.
4. **chore**: Routine tasks that don't impact code.
5. **ci**: Changes to the CI configuration.
6. **docs**: Documentation updates.
7. **style**: Changes that do not affect functionality (formatting, missing semi-colons, etc.).
8. **refactor**: Code changes that neither fix a bug nor add a feature.
9. **perf**: Changes that improve performance.
10. **test**: Adding missing tests or correcting existing ones.
11. **revert**: Reverts a previous commit.

### Example of Use:
```
fix(syntax): handle null pointer exception
```

---

## 4. Breaking Changes
A commit introducing a breaking change should either:
- Have a footer stating `BREAKING CHANGE: <description>`.
- Include an exclamation mark `!` before the colon in the type/scope.

### Example:
```
feat(auth)!: change authentication token to JWT
BREAKING CHANGE: Changed token format for authentication.
```

---

## 5. Footers and Additional Metadata
Footers provide additional context or meta-information and can contain:
- References to issues (e.g., `fixes #123`).
- A `BREAKING CHANGE` notice.

### Example with References:
```
fix: resolve race condition

Fixes #45, #76.
```

---

## 6. Using Conventional Commits with Semantic Versioning
Conventional Commits is tightly coupled with **Semantic Versioning** (SemVer). The following mappings apply:
- **fix**: increments the PATCH version.
- **feat**: increments the MINOR version.
- **BREAKING CHANGE**: increments the MAJOR version.

### Example of Versioning:
1. From `1.0.0`:
   - After a `feat`: `1.1.0`
   - After a `fix`: `1.0.1`
   - After a breaking change: `2.0.0`

---

## 7. Best Practices and Advantages
### Best Practices:
- Always use a type.
- Write clear and concise messages.
- Include footers for breaking changes.
- Use scopes for clarity.
  
### Advantages:
- Enhances communication.
- Aids in maintaining a clean project history.
- Simplifies the process of generating changelogs.

---

## 8. Assignments and Practice Exercises
### Assignment 1: Commit Message Structure
Write three commit messages based on the following scenarios:
1. You fixed a bug that caused crashing on user login.
2. You added a feature to export user data in CSV format.
3. You made a performance improvement in the data-fetching method.

### Example format:
```
fix: prevent crash on user login
```

### Assignment 2: Scenario-Based Creation
Given the following scenarios, create commit messages:
1. You are deprecating an old method of authentication.
2. You want to add a new translation feature for Spanish.
3. You updated the documentation for the new API.

### Assignment 3: Identify Types
For each of the following messages, identify the type:
1. `docs: updated the README file`
2. `revert: fix infinite loop`
3. `fix(api): handle null response error`

---

## Conclusion
Understanding Conventional Commits is crucial for maintaining a well-organized project. This structured approach to commit messages leads to better collaboration and easier project management. Make sure to complete the assignments to solidify your understanding!

In [21]:
call_gpt_for_learning(git_convention, MODEL_LLAMA)

# Mastering Conventional Commits

This course will guide you through the concepts of Conventional Commits, providing detailed explanations, examples, and assignments to master the skill of writing meaningful commit messages in your version control. This method promotes a structured and readable history, automates changelog generation, and supports semantic versioning.

## Course Overview

1. **Introduction to Conventional Commits**
   - What are Conventional Commits?
   - Benefits of using Conventional Commits.

2. **Commit Structure**
   - Understanding the commit message format.
   - Breakdown of commit types, scopes, and messages.

3. **Commit Types**
   - Detailed discussion on commit types: 
     - `feat`
     - `fix`
     - Other allowed types such as `docs`, `chore`, `style`, etc.

4. **Breaking Changes**
   - How to document breaking changes.
   - Differences between using `!` and the `BREAKING CHANGE:` footer.

5. **Footers and Body**
   - Understanding the optional body and footers.
   - Common practices for writing effective commit messages.

6. **Practical Applications**
   - Integrating with tools that support Conventional Commits.
   - Commitlint usage and setup for enforcing Conventional Commits.

7. **Best Practices and FAQs**
   - Common questions regarding commit messages.
   - Tips and practices for managing commit history.

8. **Assignments and Exercises**
   - Hands-on practice with writing commit messages.
   - Mock scenarios for applying knowledge.

---

## 1. Introduction to Conventional Commits

### What are Conventional Commits?
Conventional Commits is a specification that helps developers write readable commit messages for their version control systems. It's built on top of the existing commit message format, ensuring that teams follow a standard structure.

### Benefits of Using Conventional Commits
- **Precision in Commit History:** Allows teams to quickly understand changes with clear, structured messages.
- **Automation:** Facilitates generation of change logs and versioning based on commit types.
- **Easier Contribution:** New contributors can better navigate and contribute to projects with a clear commit history.

---

## 2. Commit Structure

### Formatting
A typical commit message is structured as follows:

```
<type>[optional scope]: <description>

[optional body]

[optional footer(s)]
```

### Breakdown
- **Type:** Must be one of the defined commit types.
- **Scope:** Optional, gives context about the change (e.g., `feat(parser)`).
- **Description:** A brief summary (imperative mood).
- **Body and Footer:** Optional sections for additional detail and information.

---

## 3. Commit Types

### Required Types
- **`feat`:** Indicates a new feature. (e.g., `feat(lang): add Polish language`)
- **`fix`:** Indicates a bug fix. (e.g., `fix(parser): fix array parsing issue`)

### Other Allowed Types
Tools like `@commitlint` recommend types such as:
- `docs`: Documentation changes.
- `style`: Code style changes (whitespace, formatting).
- `refactor`: Code refactoring that neither fixes a bug nor adds a feature.
- `test`: Adding missing tests or correcting existing tests.

---

## 4. Breaking Changes

### Documenting Breaking Changes
- Utilize `BREAKING CHANGE:` in the footer or append `!` to the type. 
- For example: 

```
feat!: send an email to the customer when a product is shipped

BREAKING CHANGE: The email will now require a customer ID.
```

### Footer vs. Exclamation Mark
Using `!` at the end of the type simplifies your commit message while also indicating a breaking change. The footer can then elaborate on the impact of this change if necessary.

---

## 5. Footers and Body

### Optional Body and Footers
- **Body:** A longer context about the changes — this must appear one blank line after the description.
- **Footers:** Additional information which may contain:
  - `BREAKING CHANGE:` descriptions.
  - References to issues or documentation.

**Example:**
```
fix: prevent racing of requests

Introduces a request id and reference to the latest request.
Dismisses responses from any but the latest request.
Round the clock testing on both ends for increased performance.

BREAKING CHANGE: Old response methods are now obsolete.
```

---

## 6. Practical Applications

### Integrating with Tools
You can leverage tools like `commitlint` to enforce Conventional Commits in your projects.

**Setup Example:**
```bash
npm install --save-dev @commitlint/config-conventional @commitlint/cli
echo "module.exports = {extends: ['@commitlint/config-conventional']};" > commitlint.config.js
```

---

## 7. Best Practices and FAQs

### Common Questions
- **Can multiple types be used?** It's encouraged to create separate commits for different types of changes.
- **What if I use the wrong commit type?** Before merging, you can use `git rebase -i` to correct your commit history.

### Tips
- Aim for clarity and brevity in descriptions.
- Maintain consistency in commit types and scopes.

---

## 8. Assignments and Exercises

### Assignment 1: Writing Commit Messages
Create 5 commit messages for the following scenarios:
1. **Feature Added:** You added a new search functionality to the application.
2. **Bug Fix:** You fixed a critical issue that caused the app to crash on startup.
3. **Documentation Update:** Updated the README to reflect recent changes.
4. **Style Changes:** Updated coding style to follow new standards.
5. **Breaking Change:** Introduced a new API endpoint, which removes an old endpoint.

### Assignment 2: Analyzing Commit Messages
Review the following commit messages and state if they follow Conventional Commits and suggest improvements:
```
- fixing a bug
- Updated the docs
- feat stuff
- BREAKING CHANGE noted here
```

### Assignment 3: Commitlint Configuration
Set up a local project to enforce Conventional Commits using `commitlint`. Show a configured `.commitlintrc.js` file and demonstrate adding a failing commit message.

---

## Conclusion
This course on Conventional Commits is aimed at empowering developers to communicate code changes effectively and leverage automation for better project management. Through structured commit messages, teams can foster better collaboration and maintain clean code history.

In [22]:
def call_model_for_links(website, model):
    llama_response = ollama.chat(
        model=model,
        messages=set_prompt_forlink(website),
        format='json'
    )

    return json.loads(llama_response['message']['content'])

relevant_links = call_model_for_links(git_convention, MODEL_MISTRAL)
print(relevant_links)

{'links': [{'type': '/en/v1.0.0', 'url': 'https://www.conventionalcommits.org/en/v1.0.0'}, {'type': '/en/v1.0.0-beta.4', 'url': 'https://www.conventionalcommits.org/en/v1.0.0-beta.4'}, {'type': '/en/v1.0.0-beta.3', 'url': 'https://www.conventionalcommits.org/en/v1.0.0-beta.3'}, {'type': '/en/v1.0.0-beta.2', 'url': 'https://www.conventionalcommits.org/en/v1.0.0-beta.2'}, {'type': '/en/v1.0.0-beta.1', 'url': 'https://www.conventionalcommits.org/en/v1.0.0-beta.1'}, {'type': '/en/v1.0.0-beta', 'url': 'https://www.conventionalcommits.org/en/v1.0.0-beta'}, {'type': '/en/', 'url': 'https://www.conventionalcommits.org/en/'}, {'type': '#summary', 'url': 'http://semver.org/#summary'}, {'type': 'http://semver.org/#specification', 'url': 'http://semver.org/#specification'}, {'type': 'https://github.com/conventional-changelog/commitlint/tree/master/%40commitlint/config-conventional', 'url': 'https://github.com/conventional-changelog/commitlint/tree/master/%40commitlint/config-conventional'}, {'type